In [1]:
import tensorflow as tf
import numpy as np
from string import punctuation
from collections import Counter
import sys

In [2]:
with open('reviews.txt', 'r') as f:
    reviews=f.read()
    
with open('labels.txt', 'r') as f:
    labels=f.read()
    


In [3]:
reviews=''.join([c for c in reviews if c not in punctuation])
split_reviews=reviews.split('\n')
full_text=' '.join(split_reviews)
words=full_text.split()
word_count=Counter(words)
vocab=sorted(word_count, key=word_count.get, reverse=True)
vocab2int={w:i for i, w in enumerate(vocab, 1)}
review_ints=[]
for review in split_reviews:
    review_ints.append([vocab2int[word] for word in review.split()])



In [4]:
labels_=labels.split('\n')
labels_int=np.array([1 if label=='positive' else 0 for label in labels_])


In [5]:
review_lens=Counter([len(x) for x in review_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))
#reviews of  length 200 is fed into network zero length review is padded with zeros and others are truncated to 200 
#characters

Zero-length reviews: 1
Maximum review length: 2514


In [6]:
seq_len=200
non_zero_idx=[i for i, r in enumerate(review_ints) if len(r)!=0]
review_ints=[review_ints[i] for i in non_zero_idx]
labels_int=[labels_int[i] for i in non_zero_idx]
features=np.zeros((len(review_ints), seq_len), dtype=int)

for idx, row in enumerate(review_ints):
    features[idx, -len(row):]=row[:seq_len]
    


In [7]:
split_fract=0.8
split_idx=int(len(features)*0.8)
train_x, val_x=features[:split_idx], features[split_idx:]
train_y, val_y=labels_int[:split_idx], labels_int[split_idx:]

test_idx=int(len(val_x)*0.5)
val_x, test_x=val_x[:test_idx], val_x[test_idx:]
val_y, test_y=val_y[:test_idx], val_y[test_idx:]

def get_batches(x, y, batch_size):
    n_batches=len(x)//batch_size
    x_, y_ = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for i in range(0, len(x), batch_size):
        yield x_[i:i+batch_size], y_[i:i+batch_size]

In [8]:
n_units=128
lstm_layer=1
batch_size=100
embed_size=300
n_words=len(vocab2int)+1 
graph=tf.Graph()

In [9]:
with graph.as_default():
    inputs=tf.placeholder(tf.int32, [None, None])
    _labels=tf.placeholder(tf.int32, [None, None])
    keep_prob=tf.placeholder(tf.float32)
    
    embed_weight=tf.Variable(tf.random_normal((n_words, embed_size), -1, 1))
    embedding=tf.nn.embedding_lookup(embed_weight, inputs)
    
    def build_cell(n_units, keep_prob):
        lstm=tf.contrib.rnn.BasicLSTMCell(n_units)
        drop=tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    batch_size_=tf.shape(inputs)[0]
    cells=tf.contrib.rnn.MultiRNNCell([build_cell(n_units, keep_prob) for _ in range(lstm_layer)])
    initial_state=cells.zero_state(batch_size_, tf.float32)
    
    output, final_state=tf.nn.dynamic_rnn(cells, embedding, initial_state=initial_state)
    
    predictions=tf.contrib.layers.fully_connected(output[:, -1],1, activation_fn=tf.sigmoid)
    cost=tf.losses.mean_squared_error(_labels, predictions)
    optm=tf.train.AdamOptimizer(learning_rate=1e-3).minimize(cost)
    
    correct_pred=tf.equal(tf.cast(tf.round(predictions), tf.int32), _labels)
    accuracy=tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    saver=tf.train.Saver()

In [14]:
epochs=3

with tf.Session(graph=graph) as sess:
    
    sess.run(tf.global_variables_initializer())
    iteration=1
    for e in range(epochs):
        #batches=get_batches(train_x, train_y, batch_size)
        state=sess.run(initial_state)
        
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
        
            feed={inputs:x,
                 _labels:np.array(y)[:, None],
                 keep_prob:0.8,
                 initial_state:state}
            
            loss, state, _ = sess.run([cost, final_state, optm], feed_dict=feed)
            
            if iteration%5==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))
                
                
            
            if iteration%25==0:
                val_acc=[]
                valid_batches=get_batches(val_x, val_y, batch_size)
                valid_state=sess.run(initial_state)
                for x1, y1 in valid_batches:
                    feed={inputs:x1,
                         _labels:np.array(y1)[:, None],
                         keep_prob:0.6,
                         initial_state:valid_state}
                    batch_acc, valid_state = sess.run([accuracy, final_state], feed_dict=feed)
                    val_acc.append(batch_acc)
                print("Val acc: {:.3f}".format(np.mean(val_acc)))
            
            iteration+=1
    
    saver.save(sess, 'checkpoints/sentiments.ckpt')


Epoch: 0/3 Iteration: 5 Train loss: 0.264
Epoch: 0/3 Iteration: 10 Train loss: 0.250
Epoch: 0/3 Iteration: 15 Train loss: 0.265
Epoch: 0/3 Iteration: 20 Train loss: 0.246
Epoch: 0/3 Iteration: 25 Train loss: 0.244
Val acc: 0.494
Epoch: 0/3 Iteration: 30 Train loss: 0.251
Epoch: 0/3 Iteration: 35 Train loss: 0.261
Epoch: 0/3 Iteration: 40 Train loss: 0.254
Epoch: 0/3 Iteration: 45 Train loss: 0.248
Epoch: 0/3 Iteration: 50 Train loss: 0.251
Val acc: 0.511
Epoch: 0/3 Iteration: 55 Train loss: 0.248
Epoch: 0/3 Iteration: 60 Train loss: 0.251
Epoch: 0/3 Iteration: 65 Train loss: 0.252
Epoch: 0/3 Iteration: 70 Train loss: 0.248
Epoch: 0/3 Iteration: 75 Train loss: 0.251
Val acc: 0.510
Epoch: 0/3 Iteration: 80 Train loss: 0.241
Epoch: 0/3 Iteration: 85 Train loss: 0.249
Epoch: 0/3 Iteration: 90 Train loss: 0.236
Epoch: 0/3 Iteration: 95 Train loss: 0.256
Epoch: 0/3 Iteration: 100 Train loss: 0.243
Val acc: 0.525
Epoch: 0/3 Iteration: 105 Train loss: 0.248
Epoch: 0/3 Iteration: 110 Train loss

In [10]:
with open('custom test dataset', 'r') as custom:
    file=custom.read()
    
def preprocess(file):
    file=''.join([c for c in file if c not in punctuation])
    split_reviews=file.split('\n')
    full_text=' '.join(split_reviews)
    words=full_text.split()
    review_ints=[]
    for review in split_reviews:
        review_ints.append([vocab2int[word] for word in review.split() if word in vocab2int])

    
    non_zero_idx=[i for i, r in enumerate(review_ints) if len(r)!=0]
    review_ints=[review_ints[i] for i in non_zero_idx]
    features=np.zeros((len(review_ints), seq_len), dtype=int)
    
    for idx, row in enumerate(review_ints):
        features[idx, -len(row):]=row[:seq_len]
    
    return features


user_test_reviews=preprocess(file)
   

In [11]:

with tf.Session(graph=graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    test_state=sess.run(cells.zero_state(batch_size, tf.float32))
    test_batches=get_batches(test_x, test_y, batch_size)
    test_acc=[]
    for x, y in test_batches:
        feed={inputs:x,
                _labels:np.array(y)[:, None],
                keep_prob:1,
                initial_state:test_state}
        batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
        test_acc.append(batch_acc)
        
    print('test accuracy : {:.3f}'.format(np.mean(test_acc)))
    
    print("***************\n")
    print("Checking model on user reviews...\n")
    
    predict=[]
    x=user_test_reviews
    values=sess.run(predictions, feed_dict={inputs:x, keep_prob:0.55})
    #print(values,"\n")
    for value in values:
        predict.append(sess.run([tf.cast(tf.round(value), tf.int32)])[0][0])
        
    for pred in predict: 
        if pred==1:
            print("Positve review")
        else:
            print("Negative review")

INFO:tensorflow:Restoring parameters from checkpoints/sentiments.ckpt
test accuracy : 0.850
***************

Checking model on user reviews...

[[0.9348978 ]
 [0.8056519 ]
 [0.48843923]
 [0.9334844 ]
 [0.52444077]
 [0.48062965]
 [0.8147987 ]
 [0.88564175]] 

[1, 1, 0, 1, 1, 0, 1, 1]
